In [7]:
import polars as pl
from dotenv import load_dotenv
import os
from datetime import datetime
load_dotenv()

True

In [8]:
# load CCOD
ccod_zip_file_name = os.getenv('HMLR_CCOD_OUTPUT_FILE')
ccod_file_root = ccod_zip_file_name.replace('.zip', '')
df_ccod =  pl.read_csv(f'downloads/{ccod_file_root}/{ccod_file_root}.csv')

In [9]:
len(df_ccod)

4181223

In [10]:
# load OS
os_files_path = os.getenv('OS_FILES_PATH')
csv_file_names = sorted(os.listdir(os_files_path))

In [11]:
record_dfs = {}
# record_types = [21,23,28,31,32] # as a min would need 28 for this
record_types = [28]

In [13]:
# read each file and assign to correct os record type
for file_name in csv_file_names:
    record_type = int(file_name[2:4])
    if record_type not in record_types: continue
    print(f'file_name: [{file_name}], time: [{datetime.now().strftime("%H:%M:%S")}]')
    temp_df = pl.read_csv(f'{os_files_path}/{file_name}', infer_schema_length=0)
    record_dfs[record_type] = temp_df

print('done')

file_name: [ID21_BLPU_Records.csv], time: [19:51:39]
file_name: [ID23_XREF_Records.csv], time: [19:51:44]
file_name: [ID28_DPA_Records.csv], time: [19:52:22]
file_name: [ID31_Org_Records.csv], time: [19:52:34]
file_name: [ID32_Class_Records.csv], time: [19:52:35]
done


In [14]:
# check everything is ok
for record_type in record_dfs.keys():
    print(f'type: [{record_type}], count: [{len(record_dfs[record_type])}]')
    
## type: [21], count: [40825714]
## type: [23], count: [197964510]
## type: [28], count: [30598226]
## type: [31], count: [1422087]
## type: [32], count: [44684909]

type: [21], count: [40825714]
type: [23], count: [197964510]
type: [28], count: [30598226]
type: [31], count: [1422087]
type: [32], count: [44684909]


In [ ]:
os.makedirs('output', exist_ok=True)
for record_type in record_dfs.keys():
    print(f'type: [{record_type}], time: [{datetime.now().strftime("%H:%M:%S")}]')
    record_dfs[record_type].write_parquet(f'output/os_recordtype_{record_type}.parquet')